In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torchtoolbox.transform as transforms
import torch.optim as optim
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from torchvision import models

In [78]:
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [3]:
class MyMobilenet (nn.Module):

    def __init__(self, mobilenet, num_class, freeze_conv=False):

        super(MyMobilenet, self).__init__()
        
        self.features = nn.Sequential(*list(mobilenet.children())[:-1])

        if freeze_conv:
            for param in self.features.parameters():
                param.requires_grad = False
        self.reducer_block = nn.Sequential(nn.Linear(1280, 128),
                                           nn.BatchNorm1d(128),
                                           nn.ReLU(),
                                           nn.Dropout(p=0.5)
                                          )
        
        self.classifier = nn.Linear(128 + 20, 6)

    def forward(self, img, meta_data=None):
        
        x = self.features(img)
        x = x.mean([2, 3])
        x = self.reducer_block(x)
        x = x.view(x.size(0), -1) # flatting
        x = torch.cat([x, meta_data], dim=1) # concatenation
        return self.classifier(x)

In [4]:
def load_model (checkpoint_path, model, opt_fn=None, loss_fn=None, epoch=None):

    if not os.path.exists(checkpoint_path):
        raise Exception ("The {} does not exist!".format(checkpoint_path))

    ckpt = torch.load(checkpoint_path)
    model.load_state_dict(ckpt['model_state_dict'])

    if opt_fn is not None and loss_fn is not None:
        opt_fn.load_state_dict(ckpt['optimizer_state_dict'])
        epoch = ckpt['epoch']
        loss_fn = ckpt['loss']
        return model, opt_fn, loss_fn, epoch
    else:
        return model

In [66]:
model = MyMobilenet(models.mobilenet_v2(pretrained=True), 6)
model.to('cpu')
loss = nn.CrossEntropyLoss()
opt_fun = optim.Adam(model.parameters(), lr = 0.0001)
path = 'E:/Results/Concatinate/Final Model/best-checkpoint/best-checkpoint.pth'  

Model = load_model(path, model,opt_fun, loss, epoch=None)
model.eval()

MyMobilenet(
  (features): Sequential(
    (0): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

In [6]:
labels = ['Atopic Dermatitis','Acne Vulgaris',  'Lichen Planus', 'Onychomycosis', 'Tinea Capitis', 'Unknown']

In [7]:
meta_data = ['Abdomain', 'Anterior_Torso','Armpit', 'Chin','Ear', 'Forehead', 'Lateral_Face',
             'Lower_Back', 'Lower_Extremity', 'Nail', 'Navel', 'Neck','Nose', 'Perioribital', 'Posterior_Torso', 'Scalp',
             'Upper_Extremity']

In [71]:
def predict(img,meta_data=None):
    
    img = Image.open(img).convert('RGB')
    img = np.asarray(img)
    #img = test_transform(img)
    img = torch.tensor(img, device=torch.device("cpu"), dtype=torch.float32)
    #img = img[None,:,:,:]
    #meta_data = torch.FloatTensor(meta_data)
    #meta_data = torch.tensor(meta_data, device=torch.device("cpu"), dtype=torch.float32)
    #meta_data = meta_data[None,:]
    inputs = (img,meta_data)
    preds = model(inputs)
    preds = torch.argmax(preds,1)
    
    #return preds
    print("The Skin Condition is: {}".format(labels[preds.max()]))

In [82]:
img = 'E:/Skin/Skin_App/Drive/Images/AD0.jpg'
img = Image.open(img).convert('RGB')
img = np.asarray(img)
img = test_transform(img)
#img = torch.tensor(img, device=torch.device("cpu"), dtype=torch.float32)
img = img.unsqueeze(0)
inputs = (img)
preds = model(inputs)

TypeError: expected Tensor as element 1 in argument 0, but got NoneType

In [57]:
img = 'E:/Skin/Skin_App/Drive/Images/AD0.jpg'
meta_data = [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 28]
k = predict(img, meta_data=None)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not tuple

In [39]:
int(meta_data)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [50]:
img = 'E:/Skin/Skin_App/Drive/Images/AD0.jpg'
#meta_data = ['Nail','M', '28']
meta_data = [int(0), int(0), int(0), int(0), int(0), int(0), int(0), int(0), int(0), 
             int(0), int(0), int(0), int(0), int(0), int(0), int(1), int(0), int(0), int(1), int(28)]
k = predict(img, meta_data)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not tuple